# DASC 5300, Fall2024, University of Texas at Arlington
# PA4 "Implementing the Go Fish Card Game in Python"


## **Academic Honesty**
This assignment must be done individually and independently. You must implement the whole assignment by yourself. Academic dishonesty is not tolerated.

## **Requirements**

1.   When you work on this assignment, you should make a copy of this notebook in Google Colab. This can be done using the option `File > Save a copy in Drive` in Google Colab.

2.  To submit your assignment, download your Colab into a .ipynb file. This
can be done using the option `Download > Download .ipynb` in Google Colab. Submit the downloaded .ipynb file/ .zip into the PA4 entry in Canvas.


## **Description**

Implementing the Go Fish card game using stacks and queues in Python. The program should simulate a game between two players, where each player draws cards from a central deck to collect sets of matching cards. The game ends when the deck is empty, and the player with the most sets at the end of the game wins. The program should use QUEUE to represent each player's hand and a STACK to represent the central deck of cards. The game should be playable in the command-line interface, with the option to specify the number of players and the number of cards dealt to each player at the start of the game."

In [4]:
import random
from collections import deque
from dataclasses import dataclass
from enum import Enum
from typing import Deque, Optional, Set, List

In [5]:
class CardRank(Enum):
    """Represent different card ranks."""
    TWO = "2"
    THREE = "3"
    FOUR = "4"
    FIVE = "5"
    SIX = "6"
    SEVEN = "7"
    EIGHT = "8"
    NINE = "9"
    TEN = "10"
    JACK = "J"
    QUEEN = "Q"
    KING = "K"
    ACE = "A"


class CardSuit(Enum):
    """Represent different card suits."""
    HEARTS = "♥"
    DIAMONDS = "♦"
    CLUBS = "♣"
    SPADES = "♠"

In [6]:
# Card class to combine rank and suit
@dataclass
class Card:
    """Represent a playing card."""
    def __init__(self, rank: CardRank, suit: CardSuit):
        """Represent a playing card.

        Args:
            rank (CardRank): Rank of the card.
            suit (CardSuit): Suit of the card.
        """
        self.rank = rank
        self.suit = suit

    def __str__(self):  # noqa: D105
        return f"{self.rank.value}{self.suit.value}"

    def __repr__(self):  # noqa: D105
        return f"Card(rank={self.rank}, suit={self.suit})"

    def __eq__(self, card):  # noqa: D105
        return self.rank == card.rank and self.suit == card.suit


In [7]:
class Player:
    """Represent a gofish player.

    A player has a hand of cards and sets collected.
    """
    def __init__(self, hand: Optional[Deque[Card]] = None):
        """Initialize a gofish player.

        Args:
            hand (Optional[Deque[Card]], optional): Hand of cards. Defaults to None.
        """
        self._hand = hand if hand else deque()
        self._sets = set()

    # Check and collect sets of 4 matching ranks
    def count_sets(self):
        """Count the number of sets the player has made."""
        rank_counts = {}
        sets = 0

        # Count the occurrences of each rank
        for card in self._hand:
            rank_counts[card.rank] = rank_counts.get(card.rank, 0) + 1

        # Identify and remove sets of 4 cards
        for rank, count in rank_counts.items():
            if count == 4:
                sets += 1
                # self._hand = deque([card for card in self._hand if card.rank != rank])
                self._sets.add(rank)

    @property
    def hand(self) -> Deque[Card]:
        """The playing hand of a player.

        Also includes the collected sets.

        Returns:
            Deque[Card]: Queue of cards representing the playing hand.
        """
        return self._hand

    @property
    def sets(self) -> Set[Card]:
        """Sets of card collected.

        Returns:
            Set[Card]: Set of all cards collected.
        """
        return self._sets

    def add_card(self, card: Card):
        """Add a new card to the hand.

        Args:
            card (Card): Playing card.
        """
        self._hand.append(card)

    def remove_cards(self, rank: CardRank) -> List[Card]:
        """Remove cards from the playing hand.

        Removes all the cards from then hand matching the rank.

        Args:
            rank (CardRank): Card Rank to remove.

        Returns:
            List[Card]: Removed cards.
        """
        removed_cards = []
        for card in list(self._hand):
            if card.rank == rank:
                removed_cards.append(card)
                self._hand.remove(card)

        return removed_cards

    def get_random_card(self) -> Card:
        """Random card from the hand.

        The card is not a part of any collected set.

        Raises:
            ValueError: If hand is empty.

        Returns:
            Card: A random card from the hand.
        """
        if not self._hand:
            raise ValueError("Cannot get a card from an empty hand")
        return random.choice([card for card in self._hand if card.rank not in self._sets])

    def __bool__(self):  # noqa: D105
        return bool(self._hand)

    def __str__(self):  # noqa: D105
        return f"Hand: {[str(x) for x in self._hand]}\nSets: {self._sets}"

In [8]:
class Simulation:
    """Go-fish! game simulation between N players with k cards.

    This simulation can shuffle cards, deal cards, and create a deck of cards.

    Progression:
        1. The created simulation begins with creating a deck of 52 cards. This deck is then
            shuffled.
        2. The shuffled deck is dealt to each of the k players limited by the cards
            per player.
        3. Turn begins with player 1.
        4. The player asks for a random card rank (provided he has at least on of them) to any
            random player.
        5. If the requested rank is found, the turn continues and player asks for another random
            card from a random player.
        6. If the requested rank is not avaialable, the player is told Go Fish! and the player draws
            a card from the deck and the turn is passed to the next player.
        7. The game continues until the deck is exhausted upon which the number of sets collected by
            each player is counted.
        8. The player with the most collected sets wins.
    """
    def __init__(self, num_players: int = 2, cards_per_player: int = 7):
        """Initializes the go-fish simulation.

        Args:
            num_players (int, optional): Number of players. Defaults to 2.
            cards_per_player (int, optional): Initial cards per player. Defaults to 7.
        """
        self.num_players = num_players
        self.cards_per_player = cards_per_player
        self.winner = None

    # Create a full deck
    def create_deck(self) -> Deque[Card]:
        """Create a deck of cards.

        Returns:
            Deque[Card]: Deck of card represented as a stack.
        """
        return deque([Card(rank, suit) for rank in CardRank for suit in CardSuit])

    # Shuffle the deck
    def shuffle_deck(self, deck: Deque[Card]) -> Deque[Card]:
        """Shuffle a deck of cards.

        Args:
            deck (Deque[Card]): Stack of cards to be shuffled.

        Returns:
            Deque[Card]: Shuffled deck of cards.
        """
        temp_list = list(deck)
        random.shuffle(temp_list)
        return deque(temp_list)

    # Deal cards to players
    def deal_cards(self, deck: Deque[Card]) -> Deque[Card]:
        """Deal cards to each player.

        The cards are dealt iteratively to each player.

        Args:
            deck (Deque[Card]): Deck of cards.

        Returns:
            Deque[Card]: Hands of dealt cards.
        """
        hands = [deque() for _ in range(self.num_players)]
        for _ in range(self.cards_per_player):
            for hand in hands:
                if deck:  # Check if the deck is not empty
                    hand.append(deck.popleft())  # Deal one card from the deck

        return hands

    def random_excluding(self, number: int, exclude: int) -> int:
        """Generate a random number from a range.

        Exclusive of the specified number.

        Args:
            number (int): Range of numbers to generate.
            exclude (int): Exclusion number.

        Returns:
            int: A random number from the specified range.
        """
        # Create a list excluding the `exclude` number
        choices = [i for i in range(number) if i != exclude]
        # Randomly select a number from the filtered list
        return random.choice(choices)

    def run(self):
        """Run the simulation."""
        deck: Deque[Card] = self.create_deck()
        deck = self.shuffle_deck(deck)

        players = [Player(hand=hand) for hand in self.deal_cards(deck=deck)]

        for index, player in enumerate(players):
            print(f"Player {index+1}")
            print(str(player))

        turn = 0
        while deck:
            current_player_idx = turn % self.num_players
            current_player = players[current_player_idx]
            current_player.count_sets()

            print(f"\nCards left in Deck: {len(deck)}")

            print(f"\nPlayer {current_player_idx + 1}'s turn.")
            print(str(current_player))

            if len(current_player.sets) * 4 == len(current_player.hand):
                current_player.add_card(deck.popleft())

            target_player_index = self.random_excluding(self.num_players, current_player_idx)
            target_player = players[target_player_index]

            requested_rank = current_player.get_random_card().rank
            print(f"Asking for {requested_rank.value} from Player {target_player_index + 1}")

            removed_cards = target_player.remove_cards(requested_rank)

            if not removed_cards:
                print("Go Fish!")
                current_player.add_card(deck.popleft())
                turn += 1

            for card in removed_cards:
                current_player.add_card(card)

            print(f"Got cards: {[str(card) for card in removed_cards]}")
        else:
            for player in players:
                player.count_sets()

        # Determine winner
        print("\nGame over!")
        player_sets = []
        for i, player in enumerate(players):
            player_sets.append(len(player.sets))
            print(f"Player {i + 1} collected {len(player.sets)} set(s).")
            print([rank.value for rank in player.sets])

        max_sets = max(player_sets)
        winners = [i + 1 for i, sets in enumerate(player_sets) if sets == max_sets]

        if len(winners) > 1:
            print(f"It's a tie between players {', '.join(map(str, winners))}!")
        else:
            print(f"Player {winners[0]} wins!")


In [9]:
sim = Simulation(cards_per_player=20)
sim.run()

Player 1
Hand: ['4♥', '9♦', '5♣', 'Q♣', '4♦', 'K♦', 'A♣', 'K♣', '2♥', '10♣', '9♥', '7♦', '5♦', 'K♠', '2♦', '9♠', '3♦', '6♦', 'A♠', 'Q♥']
Sets: set()
Player 2
Hand: ['8♥', '2♣', 'J♣', '3♠', '3♥', 'K♥', '8♦', '7♥', '8♣', '7♣', '5♥', 'A♥', 'J♠', '6♣', 'J♦', 'J♥', '10♦', '6♠', '8♠', '10♠']
Sets: set()

Cards left in Deck: 12

Player 1's turn.
Hand: ['4♥', '9♦', '5♣', 'Q♣', '4♦', 'K♦', 'A♣', 'K♣', '2♥', '10♣', '9♥', '7♦', '5♦', 'K♠', '2♦', '9♠', '3♦', '6♦', 'A♠', 'Q♥']
Sets: set()
Asking for 9 from Player 2
Go Fish!
Got cards: []

Cards left in Deck: 11

Player 2's turn.
Hand: ['8♥', '2♣', 'J♣', '3♠', '3♥', 'K♥', '8♦', '7♥', '8♣', '7♣', '5♥', 'A♥', 'J♠', '6♣', 'J♦', 'J♥', '10♦', '6♠', '8♠', '10♠']
Sets: {<CardRank.JACK: 'J'>, <CardRank.EIGHT: '8'>}
Asking for A from Player 1
Got cards: ['A♣', 'A♠']

Cards left in Deck: 11

Player 2's turn.
Hand: ['8♥', '2♣', 'J♣', '3♠', '3♥', 'K♥', '8♦', '7♥', '8♣', '7♣', '5♥', 'A♥', 'J♠', '6♣', 'J♦', 'J♥', '10♦', '6♠', '8♠', '10♠', 'A♣', 'A♠']
Sets: {<Card